In [12]:
#Testando o o que será colocado em threads_cam.py


import face_recognition as fr
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import pandas as pd
import sklearn
import math
import pickle #Estudar sobre
from flask import Flask, render_template, Response



def resizeImage(image, **kwargs):
    if 'scale' in kwargs.keys():
        scale = kwargs.get('scale')
        x = int(image.shape[0] * scale)
        y = int(image.shape[1] * scale)
    return cv2.resize(image, dsize=(x, y), interpolation=cv2.INTER_AREA)

def compare_images(im1, im2):
    plt.Figure(figsize=(10, 10))
    plt.subplot(121)
    plt.imshow(im1)
    plt.subplot(122)
    plt.imshow(im2)

def load_image(path, gray=True):
    im = cv2.imread(path)
    if gray: im = cv2.cv2tColor(im, cv2.COLOR_BGR2RGB)
    return im

def flat_images(images):
    #Flatten the images array
    n, m = images[0].shape
    k = len(images)
    flaten_images = np.zeros((n * m, k), dtype='uint8').T
   
    for i in range(k):
        flaten_images[i] = images[i].flatten()
    return flaten_images


def plotSample(images, labels):
    size = len(images)
    plot_size = np.array(list((3,4))) * 4
    plt.figure(figsize=tuple(plot_size))
    for i in range(12):
        plt.subplot(3,4, i+1)
        plt.imshow(images[i])
        plt.yticks([])
        plt.xticks([])
        plt.xlabel(labels[i])
    plt.show()
   

# MUST BE FIXED
def get_label(frame, candidates, tolerance):
    global df
    name = None

    candidates_subset = df.iloc[candidates[0]]
    distances = []
    for encode in candidates_subset['Face Encoding']:
        distances.append(np.linalg.norm(frame - encode[0]))

    count = len([dst for dst in distances if dst > tolerance])
    closest_distance = np.amin(distances)

    print(closest_distance)
    if count > 2 and closest_distance > tolerance:
        name = "Unknow"
  
    index = np.argmin(distances)
    guess_name = candidates_subset.iloc[index]['Name']
    print(guess_name)

    return [name], count

def read_dir(directory, for_one=False, n_images=4):
    path = []
    images = []
    data = []
    
    if not(for_one):
        for sub_folder in os.listdir(directory):
            for n_picture, filename in enumerate(os.listdir(os.path.join(directory, sub_folder))):
                image_path = directory+'/'+sub_folder+'/'+filename
                data.append({"Name": sub_folder, 'Path': image_path})
                if n_picture > n_images:
                    break
    else:
        for image in os.listdir(directory):
            image_path = os.path.join(directory,image)
            data.append({"Name": directory.split('/').pop(), "Path":image_path})            
     
    df = pd.DataFrame(data)
    return df

def read_image(path):
    images = []
    for x in path:
        im = cv2.imread(x)
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        images.append(im)
    return images


def knn_train(X_train, y_train, model_save_path=None, threads=-1):
    from sklearn.neighbors import KNeighborsClassifier
    n_neighbors = int(math.sqrt(len(X_train))) #Sera susbtituído pelo elbow method
    
    model = KNeighborsClassifier(n_neighbors= n_neighbors, weights='distance', algorithm='auto', p=2, metric='minkowski', n_jobs = threads)
    model.fit(X_train, y_train)
    if model_save_path:
        save_binary(model, model_save_path)
    print("End of training")
    
    return model

def knn_predict(image, model_path=None, verbose=False):
    if model_path:
        with open(model_path, 'rb') as f:
            model = pickle.load(f)
   
    y_pred = model.kneighbors(image, n_neighbors=1)
   
    return y_pred

def save_binary(pkl_file, path):
    with open(path, 'wb') as f:
        pickle.dump(pkl_file, f)

def load_binary(path):
    with open(path, 'rb') as f:
        pkl_file = pickle.load(f)
    return pkl_file

  
def first_train(train_dir, model_save_path=None, df_save_path=None):
    df = read_dir(train_dir)
    df = df.sort_values(["Name"]) #@@@ Here because of the Dataset @@@
    df = df.iloc[:4500, :] #@@@ Here because of the Dataset @@@
    df['Image'] = read_image(df['Path'])
    print(df.tail(15))
    print("End reading")

    print("Start Location")
    df["Face Location"] = df["Image"].apply(lambda x: fr.face_locations(x, 2, model="hog"))
    print("End Location")
    
    print("Start Encoding")
    df["Face Encoding"] = df.apply(lambda x: fr.face_encodings(x["Image"], x["Face Location"]), axis=1)
    print("End Encoding")
    df = df[df.apply(lambda x: len(x['Face Encoding']) == 1, axis= 1)] #@@@@ Here because of the Dataset @@@
    if df_save_path:
        save_binary(df, df_save_path) 
    print(df.head(10))

    #Encode array has to be a 'pure' array to sklearn train
    # I won't find another way to do with thease piece of code maybe i am dumb
    X_train = []
    for elem in df['Face Encoding'].values:
        X_train.append(elem[0]) 

    model = knn_train(X_train, df['Name'].values, model_save_path)
    print("End of first train")
    return model, df

model_save_path = "../../knn_model.clf"
train_dir = "../../archive/lfw-deepfunneled"
df_save_path = '../../bkp/dataset_example.pkl'

print("Import sem erros")
    


Import sem erros


In [11]:
model, df = first_train(train_dir, model_save_path) #será executado
# Abaixo são testes

FileNotFoundError: [Errno 2] No such file or directory: '../archive/lfw-deepfunneled'

In [26]:
pd.to_pickle(df, df_save_path)

In [13]:
#Se já definidos
model = load_binary(model_save_path)
df = pd.read_pickle(df_save_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1586 entries, 2469 to 13197
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name           1586 non-null   object
 1   Path           1586 non-null   object
 2   Image          1586 non-null   object
 3   Face Location  1586 non-null   object
 4   Face Encoding  1586 non-null   object
dtypes: object(5)
memory usage: 74.3+ KB


In [5]:
df["Name"].values

array(['Don_Siegelman', 'Donald_Anderson', 'Donald_Carty', ...,
       'Gloria_Macapagal_Arroyo', 'Gloria_Macapagal_Arroyo',
       'Gloria_Macapagal_Arroyo'], dtype=object)

In [22]:
unk_encode = df['Face Encoding'].iloc[563]
unk_loc = df['Face Location'].iloc[563]
df['Name'].iloc[563]

'Felipe_Rezende'

In [22]:
y_knn = model.kneighbors(n_neighbors=9)
y_knn[1][1]
df.index


Int64Index([ 2469,  8652,  8849,  8249,  8248,  7252,  7253,  7254,  7255,
             5806,
            ...
             3150, 13233, 13202, 13224, 13223, 13201, 13195, 13237, 13196,
            13197],
           dtype='int64', length=1586)

In [9]:
y_pred = model.predict(unk_encode)
y_pred

array(['Donald_Rumsfeld'], dtype=object)

In [9]:
kk = df[df['Name'] == "Aaron_Eckhart"]
kk

,Name,Path,Image,Face Location,Face Encoding


In [12]:

df_2 = read_dir(train_dir, retrieve_one_image=True)
# df_2 = df_2.iloc[:1500, :] #Here because of the Dataset 
# df_2['Image'] = read_image(df_2['Path'])
# print(df_2['Name'].head(10))

In [20]:
# kk = df_2[df_2['Name'] == "AFelipe_Rezende"]
# kk
df_3 = df_2.sort_values(["Name"])
df_3.sort_index

<bound method DataFrame.sort_index of                     Name                                               Path
3985     AFelipe_Rezende     archive/lfw-deepfunneled/AFelipe_Rezende/1.jpg
3938             AJ_Cook  archive/lfw-deepfunneled/AJ_Cook/AJ_Cook_0001.jpg
2612            AJ_Lamas  archive/lfw-deepfunneled/AJ_Lamas/AJ_Lamas_000...
4259       Aaron_Eckhart  archive/lfw-deepfunneled/Aaron_Eckhart/Aaron_E...
4671         Aaron_Guiel  archive/lfw-deepfunneled/Aaron_Guiel/Aaron_Gui...
...                  ...                                                ...
3950      Zorica_Radovic  archive/lfw-deepfunneled/Zorica_Radovic/Zorica...
4177      Zulfiqar_Ahmed  archive/lfw-deepfunneled/Zulfiqar_Ahmed/Zulfiq...
4094        Zumrati_Juma  archive/lfw-deepfunneled/Zumrati_Juma/Zumrati_...
5698     Zurab_Tsereteli  archive/lfw-deepfunneled/Zurab_Tsereteli/Zurab...
1184  Zydrunas_Ilgauskas  archive/lfw-deepfunneled/Zydrunas_Ilgauskas/Zy...

[5750 rows x 2 columns]>

In [21]:
df_3.head(30)

,Name,Path
3985,AFelipe_Rezende,archive/lfw-deepfunneled/AFelipe_Rezende/1.jpg
3938,AJ_Cook,archive/lfw-deepfunneled/AJ_Cook/AJ_Cook_0001.jpg
2612,AJ_Lamas,archive/lfw-deepfunneled/AJ_Lamas/AJ_Lamas_000...
4259,Aaron_Eckhart,archive/lfw-deepfunneled/Aaron_Eckhart/Aaron_E...
4671,Aaron_Guiel,archive/lfw-deepfunneled/Aaron_Guiel/Aaron_Gui...
4134,Aaron_Patterson,archive/lfw-deepfunneled/Aaron_Patterson/Aaron...
1454,Aaron_Peirsol,archive/lfw-deepfunneled/Aaron_Peirsol/Aaron_P...
1920,Aaron_Pena,archive/lfw-deepfunneled/Aaron_Pena/Aaron_Pena...
2177,Aaron_Sorkin,archive/lfw-deepfunneled/Aaron_Sorkin/Aaron_So...
1095,Aaron_Tippin,archive/lfw-deepfunneled/Aaron_Tippin/Aaron_Ti...


In [21]:
df["Face Location"] = df["Image"].apply(lambda x: fr.face_locations(x, 2, model="hog"))
df["Face Encoding"] = df.apply(lambda x: fr.face_encodings(x["Image"], x["Face Location"]), axis=1)
df.to_pickle(df_save_path) #parquet does not support 2D arrays
print("Fim encode")


Fim encode


In [40]:
df.to_pickle(df_save_path)

In [56]:
df = pd.read_pickle(df_save_path)
df = df[df.apply(lambda x: len(x['Face Encoding']) == 1, axis= 1)]

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1405 entries, 0 to 1499
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name           1405 non-null   object
 1   Path           1405 non-null   object
 2   Image          1405 non-null   object
 3   Face Location  1405 non-null   object
 4   Face Encoding  1405 non-null   object
dtypes: object(5)
memory usage: 65.9+ KB


In [58]:
#Removing background faces
# sum_list_loc_m = []
# result = zip(df_2['Face Location'][:13], df_2['Face Encoding'][:13])
# for face_location, face_encoding in result:
#     if len(face_location) > 1:
#         mydict = dict(zip(face_location, face_encoding))
#         sorted_dict = dict()
#         for key in sorted(mydict, key=lambda x: sum(x), reverse=True):
#             sorted_dict.update({key: mydict[key]})

# first_item = next(iter(mydict_2.items()))
# first_face_location, first_face_encoding = first_item
     

In [60]:
uncaps_encode = []
for elem in df['Face Encoding'].values:
    uncaps_encode.append(elem[0]) 
    


In [66]:
#Devo retornar um array com as posições de cada face e seus nomes para que o draw possa iterar neste, mesmo que seja unitário
#No momento do cadastro a screenshot deverá ser validada, para que se armazene somente a foto que houver apenas uma face
model = knn_train(uncaps_encode, df['Name'])
y_pred = model.predict(unk_encode)
print(y_pred)
# y_pred = model.predict(frame_encoded)
# candidate = df[df['Name'] == y_pred]
# face_distance = can


End of training
['Tom_Watson']


In [28]:
#GET LABELS and verify face distance

candidates = np.array([[563, 565, 564, 763, 567, 528]])
candidates_subset = df.iloc[candidates[0]]
distances = []
for encode in candidates_subset['Face Encoding']:
    distances.append(np.linalg.norm(unk_encode[0] - encode[0]))

count = len([dst for dst in distances if dst > 0.5])
count



1

In [16]:
df['Name'].tail(10)

3150               Gloria_Gaynor
13233    Gloria_Macapagal_Arroyo
13202    Gloria_Macapagal_Arroyo
13224    Gloria_Macapagal_Arroyo
13223    Gloria_Macapagal_Arroyo
13201    Gloria_Macapagal_Arroyo
13195    Gloria_Macapagal_Arroyo
13237    Gloria_Macapagal_Arroyo
13196    Gloria_Macapagal_Arroyo
13197    Gloria_Macapagal_Arroyo
Name: Name, dtype: object